# Generate job files for MSI computing
Main feature is to get all subfolders in a directory, and set up the output folders correspondingly

In [2]:
import myImageLib as mil
import os
import numpy as np

In [3]:
def header(walltime, mem, code_path):
    # hrs, gb
    print('#!/bin/bash -l')
    print('#PBS -l walltime={0}:00:00,nodes=1:ppn=8,mem={1}gb'.format(walltime, mem))
    print('#PBS -m abe')
    print('#PBS -M liux3141@umn.edu\n')
    print('cd {}'.format(code_path))
    print('module load python3')
    print('source activate pythonEnv')

In [8]:
# job file generator
header(8, 3, '~/code/Python/Correlation/src/py_files')
conc = [20, 40, 60, 80]
num = [1, 2, 3]
for c in conc:
    for n in num:
        # print('python div2.py ~/Data/02042020/piv_result_50/{0:d}-{1:d} ~/Data/02042020/{0:d}-{1:d}/8-bit ~/Data/02042020/div/{0:d}-{1:d} 50 25 &'.format(c, n))
        # print('python div_x_dcadv.py ~/Data/02042020/div/{0:d}-{1:d} ~/Data/02042020/dc_adv/{0:d}-{1:d} ~/Data/02042020/divXdcadv/{0:d}-{1:d} &'.format(c, n))
        print('python df2_nobp.py ~/Data/02042020/{0:2d}-{1:d}/8-bit ~/Data/02042020/df2_nobp/{0:2d}-{1:d}'.format(c, n))
conc = [20, 40, 60, 80, 100]
for c in conc:
    print('python df2_nobp.py ~/Data/Wei/Dynamics_raw/{0:d} ~/Data/Wei/Dynamics_raw/df2_nobp/{0:d}'.format(c))
print('python df2_nobp.py ~/Data/03032020/10n0/8-bit ~/Data/03032020/df2_nobp/10n0')
print('wait')

#!/bin/bash -l
#PBS -l walltime=8:00:00,nodes=1:ppn=8,mem=3gb
#PBS -m abe
#PBS -M liux3141@umn.edu

cd ~/code/Python/Correlation/src/py_files
module load python3
source activate pythonEnv
python df2_nobp.py ~/Data/02042020/20-1/8-bit ~/Data/02042020/df2_nobp/20-1
python df2_nobp.py ~/Data/02042020/20-2/8-bit ~/Data/02042020/df2_nobp/20-2
python df2_nobp.py ~/Data/02042020/20-3/8-bit ~/Data/02042020/df2_nobp/20-3
python df2_nobp.py ~/Data/02042020/40-1/8-bit ~/Data/02042020/df2_nobp/40-1
python df2_nobp.py ~/Data/02042020/40-2/8-bit ~/Data/02042020/df2_nobp/40-2
python df2_nobp.py ~/Data/02042020/40-3/8-bit ~/Data/02042020/df2_nobp/40-3
python df2_nobp.py ~/Data/02042020/60-1/8-bit ~/Data/02042020/df2_nobp/60-1
python df2_nobp.py ~/Data/02042020/60-2/8-bit ~/Data/02042020/df2_nobp/60-2
python df2_nobp.py ~/Data/02042020/60-3/8-bit ~/Data/02042020/df2_nobp/60-3
python df2_nobp.py ~/Data/02042020/80-1/8-bit ~/Data/02042020/df2_nobp/80-1
python df2_nobp.py ~/Data/02042020/80-2/8-bit ~/Data